In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("C:\\Users\\user\\Downloads\\Amazon Sale Report.csv\\Amazon Sale Report.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_13472\369179484.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("C:\\Users\\user\\Downloads\\Amazon Sale Report.csv\\Amazon Sale Report.csv")


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

In [4]:

def preprocess_orders(data: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess the orders dataset:
    - Drop irrelevant or high-cardinality columns
    - Rename columns if needed
    - Extract useful features from 'Date'
    - Create derived features like day, month, weekday, weekend flag
    """
    
    # 1. Drop unwanted columns if they exist
    DROP_COLS = [
        "index",
        "Order ID",           # pure identifier
        "SKU",                # high cardinality ID
        "promotion-ids",      # sparse / leakage-prone
        "Unnamed: 22",
        "ASIN",
        "fulfilled-by",
        "ship-postal-code",
        "currency",
        "Size"
    ]
    
    data = data.drop(columns=[c for c in DROP_COLS if c in data.columns])
    
    # 2. Rename columns if necessary
    if 'Sales Channel ' in data.columns:
        data.rename(columns={'Sales Channel ':'Sales Channel'}, inplace=True)
    
    # 3. Convert 'Date' to datetime safely
    if 'Date' in data.columns:
        data["Date"] = pd.to_datetime(data["Date"], dayfirst=True, errors="coerce")
        
        # Extract date-related features
        data["order_day"] = data["Date"].dt.day_name()
        data["order_month"] = data["Date"].dt.month_name()
        data["order_weekday"] = data["Date"].dt.weekday
        data["is_weekend"] = data["order_weekday"].isin([5,6]).astype(int)
        
        # Drop original Date column
        data = data.drop(columns=["Date"])
    
    return data


In [5]:
data = preprocess_orders(data)
print(data.head())

C:\Users\user\AppData\Local\Temp\ipykernel_13472\103245547.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Date"] = pd.to_datetime(data["Date"], dayfirst=True, errors="coerce")


                         Status Fulfilment Sales Channel ship-service-level  \
0                     Cancelled   Merchant     Amazon.in           Standard   
1  Shipped - Delivered to Buyer   Merchant     Amazon.in           Standard   
2                       Shipped     Amazon     Amazon.in          Expedited   
3                     Cancelled   Merchant     Amazon.in           Standard   
4                       Shipped     Amazon     Amazon.in          Expedited   

     Style       Category Courier Status  Qty  Amount    ship-city  \
0   SET389            Set            NaN    0  647.62       MUMBAI   
1  JNE3781          kurta        Shipped    1  406.00    BENGALURU   
2  JNE3371          kurta        Shipped    1  329.00  NAVI MUMBAI   
3    J0341  Western Dress            NaN    0  753.33   PUDUCHERRY   
4  JNE3671            Top        Shipped    1  574.00      CHENNAI   

    ship-state ship-country    B2B order_day order_month  order_weekday  \
0  MAHARASHTRA           IN  

### Feature Engineering

In [6]:
# seperating numerical and categorical columns
numerical_cols = data.select_dtypes(include="number").columns
print("numerical_cols:\n", numerical_cols)
print()

categorical_cols = data.select_dtypes(exclude="number").columns
print("catgeorical_cols:\n", categorical_cols)

numerical_cols:
 Index(['Qty', 'Amount', 'order_weekday', 'is_weekend'], dtype='object')

catgeorical_cols:
 Index(['Status', 'Fulfilment', 'Sales Channel', 'ship-service-level', 'Style',
       'Category', 'Courier Status', 'ship-city', 'ship-state', 'ship-country',
       'B2B', 'order_day', 'order_month'],
      dtype='object')


In [7]:
numerical_cols = numerical_cols.drop(['Amount'])
numerical_cols

Index(['Qty', 'order_weekday', 'is_weekend'], dtype='object')

In [8]:
# ml preprocessing pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from category_encoders.count import CountEncoder

def build_tree_preprocessor(num_cols, cat_cols):
    
    num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median"))
        # no scaling for trees
    ])

    cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", CountEncoder(normalize=True))  # frequency encoding
    ])

    return ColumnTransformer([
        ("num", num_pipeline, num_cols),
        ("cat", cat_pipeline, cat_cols)
    ])


In [9]:
# for linear models -> scaling is on!
preprocessor = build_tree_preprocessor(
    num_cols=numerical_cols,
    cat_cols=categorical_cols,
)

In [10]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 Index(['Qty', 'order_weekday', 'is_weekend'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  CountEncoder(combine_min_nan_groups=True,
                                                               normalize=True))]),
                                 Index(['Status', 'Fulfilment', 'Sales Channel', 'ship-service-level', 'Style',
       'Category', 'Courier Status', 'ship-city', 'ship-state', 'ship-country',
       'B2B', 'order_day', 'order_month'],
      dtype='object'))])

In [11]:

# seperating the input and target variable from data
TARGET = "Amount"   # regression target

X = data.drop(columns=[TARGET])
y = data[TARGET]


In [12]:
# fill target with median values
y.fillna(value=data['Amount'].median(), inplace=True)


In [13]:
X.shape

(128975, 16)

In [14]:
y.shape

(128975,)

In [15]:
# seperating the input and target data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)


In [16]:
X_train.shape

(103180, 16)

In [17]:
y_train.shape

(103180,)

In [18]:
# apply preprocessor and take the sample records

def process_and_sample(X_train, X_test, y_train, sample_frac=0.1, n_bins=10, preprocessor=None, random_state=42):
    # 1. Fit preprocessor on full X_train
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)
    
    # 2. Create quantile bins for stratified sampling
    y_bins = pd.qcut(y_train, q=n_bins, duplicates="drop")
    
    # 3. Stratified sampling
    X_train_sampled, _, y_train_sampled, _ = train_test_split(
        X_train_processed,
        y_train,
        test_size=1-sample_frac,   # keep fraction for training
        stratify=y_bins,
        random_state=random_state
    )
    
    return X_train_processed, X_test_processed, X_train_sampled, y_train_sampled


In [34]:
X_train_processed, X_test_processed, X_train_sampled, y_train_sampled = process_and_sample(
    X_train, X_test, y_train, sample_frac=0.25, preprocessor=preprocessor
)

print("Full X_train_processed shape:", X_train_processed.shape)
print("Sampled X_train_sampled shape:", X_train_sampled.shape)


Full X_train_processed shape: (103180, 16)
Sampled X_train_sampled shape: (25795, 16)


### Model Training

In [35]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [36]:
gbr = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=7,
    subsample=0.8,
    random_state=42
)

gbr.fit(X_train_sampled, y_train_sampled)


GradientBoostingRegressor(learning_rate=0.05, max_depth=7, n_estimators=300,
                          random_state=42, subsample=0.8)

In [37]:
y_pred = gbr.predict(X_test_processed)


In [38]:

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2:", r2)


MAE: 86.62740426916297
MSE: 21417.469651033836
RMSE: 146.34708624032743
R2: 0.7171429462091137


now,using ***GridSearchCV*** to find the best model and best params

In [40]:
from sklearn.model_selection import GridSearchCV


In [42]:
gbr = GradientBoostingRegressor(
    random_state=42
)

gbr

GradientBoostingRegressor(random_state=42)

In [43]:
param_grid = {
    "n_estimators": [200, 300, 500],
    "learning_rate": [0.03, 0.05, 0.1],
    "max_depth": [3, 4, 5],
    "subsample": [0.7, 0.8, 1.0],
    "min_samples_leaf": [1, 5, 10]
}


In [ ]:
grid_search = GridSearchCV(
    estimator=gbr,
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",
    cv=3,
    n_jobs=-1,
    verbose=2
)


In [ ]:
# initiating the training process
grid_search.fit(X_train_sampled, y_train_sampled)


In [ ]:

# fining the best model and parameters
best_gbr = grid_search.best_estimator_

print("Best Parameters:")
print(grid_search.best_params_)



In [ ]:
y_pred = best_gbr.predict(X_test_processed)


In [ ]:
# evaulate on full dataset
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2:", r2)
